In [1]:
import numpy as np
import pandas as pd

import torch
from torch import nn
from torch.distributions import constraints
import functools

import pyro
import pyro.distributions as dist
from pyro.infer import SVI, JitTraceEnum_ELBO, TraceEnum_ELBO
from pyro.contrib.autoguide import AutoDiagonalNormal, AutoMultivariateNormal, AutoGuideList, AutoDelta
from pyro.optim import ClippedAdam

import time
# fix MCMCndom generator seed (for reproducibility of results)
np.random.seed(42)

In [2]:
import requests, io
r = requests.get('https://github.com/MikkelGroenning/MBML_project/blob/master/notebook/fakedata.npy?raw=true')

data = np.load(io.BytesIO(r.content)).astype('int32')

In [8]:
data_torch = torch.tensor(data).long()
data_torch = data_torch.view(25,50,100).long() #docs, words, t
data_torch.shape

NUM_TOPICS = 5
NUM_WORDS = 125
NUM_WORDS_PER_DOCUMENT = data_torch.shape[1]   
NUM_DOCS = data_torch.shape[0]         
T = data_torch.shape[2]

psi = torch.ones((NUM_DOCS, NUM_TOPICS-1))
delta = torch.ones(NUM_TOPICS-1)
sigma = torch.ones(NUM_WORDS_PER_DOCUMENT)/NUM_WORDS_PER_DOCUMENT

In [43]:
def model(T=100, data=None, output=True):
    """ Define priors over beta1, beta2, tau, noises, sigma, z_prev1 and z_prev2 (keep the shapes in mind)
    # Your code here
    """

    ALPHA = [] 

    BETA = []
    
    for t in range(T):
        with pyro.plate("topics_%d"%t, NUM_TOPICS):
            if t == 0:
                beta_t = pyro.sample(name="beta_%d"%t, fn=dist.Normal(loc=0, scale=sigma).to_event(1))
            else:
                beta_t = pyro.sample(name="beta_%d"%t, fn=dist.Normal(loc=beta_prev, scale=sigma).to_event(1))

            BETA.append(beta_t)
            beta_prev = beta_t 
        
        if t == 0:
            alpha_t = pyro.sample(name="alpha_%d"%t, fn=dist.Normal(loc = torch.zeros(NUM_TOPICS-1), scale = delta).to_event(1))
        else:
            alpha_t = pyro.sample(name="alpha_%d"%t, fn=dist.Normal(loc = alpha_prev, scale = delta).to_event(1))
        
        ALPHA.append(alpha_t)
        alpha_prev = alpha_t

        with pyro.plate("documents_%d"%t, NUM_DOCS) as doc_ind:
            if data is not None:
                with pyro.util.ignore_jit_warnings():
                    assert data.shape == (NUM_DOCS, NUM_WORDS_PER_DOCUMENT, T)
                data_p = data[doc_ind, :, t]

            theta_t = pyro.sample("theta_%d"%t, dist.LogisticNormal(
                loc = alpha_t,
                scale = psi
            ))
            
            with pyro.plate("words_%d"%t, NUM_WORDS_PER_DOCUMENT): # as word_ind:
                z = pyro.sample("z_%d"%t, dist.Categorical(probs = torch.nn.functional.softmax(theta_t, 0)), infer={"enumerate": "sequential"}).type(torch.LongTensor)
                w = pyro.sample("w_%d"%t, dist.Categorical(probs = torch.nn.functional.softmax(beta_t[z], 0)), obs=data_p)
                if((output==True) & (t==99)):
                    print(w)
                    print(z)
                    print('Beta param :', beta_t.shape)
                    print('alpha param :', alpha_t.shape)
                    print('theta_t param :', theta_t.shape)
                    print('W shape: ', w.shape)
                    print('z shape: ', z.shape)
    return z, w

In [44]:
pyro.clear_param_store()
def guide(T=100, data=None):
    psi_q = pyro.param(
        "psi_q",
         torch.ones(1),
         constraint = constraints.positive
    )
    delta_q = pyro.param(
        "delta_q",
         torch.ones(NUM_TOPICS-1),
         constraint = constraints.positive
    )
    sigma_q = pyro.param(
        "sigma_q", 
        torch.ones(NUM_WORDS_PER_DOCUMENT)/NUM_WORDS_PER_DOCUMENT,
        constraint = constraints.positive
    )
    alpha_q = pyro.param(
        'alpha_q', 
        lambda: torch.zeros(T, 4)
    )
    
    for t in range(T):
        pyro.sample('alpha_%d'%t, dist.Normal(loc = alpha_q[t,:], scale = delta_q))
        
        beta_q = pyro.param(
            'beta_q', 
            lambda: torch.zeros((T, 50)),
            constraint = constraints.simplex
        )
        
        with pyro.plate("topics_%d"%t, NUM_TOPICS):
            pyro.sample('beta_%d'%t, dist.Normal(loc=beta_q[t,:], scale=sigma_q).to_event(1))

        theta_q = pyro.param(
            "theta_q%d"%t,
            lambda: torch.ones(T, 25, 4),
            constraint = constraints.simplex
        )
        
        with pyro.plate("documents_%d"%t, NUM_DOCS):
            pyro.sample('theta_%d'%t, dist.LogisticNormal(loc = theta_q[t, :,:], scale=psi_q[0:4])) 

elbo = TraceEnum_ELBO(max_plate_nesting=2)

optim = ClippedAdam({'lr': 0.005})
svi = SVI(model, guide, optim, elbo)

# Define the number of optimization steps
n_steps = 400

# Do gradient steps
for step in range(n_steps):
    elbo = svi.step(T=100, data=data_torch)
    if step % 100 == 0:
        print("[%d] ELBO: %.1f" % (step, elbo))

tensor([[ 80,  84,  66,  ..., 112, 100,  52],
        [ 50,  14, 114,  ..., 113,  66,  12],
        [ 25,  21,  75,  ...,  39,  27,  63],
        ...,
        [ 63,  78,  14,  ...,  18,  92,  94],
        [ 51,  89, 109,  ...,  65,  79,  14],
        [ 88,  87,  77,  ...,  63,  91,  83]])
tensor([[2, 2, 0,  ..., 3, 1, 2],
        [4, 3, 2,  ..., 0, 0, 3],
        [1, 3, 3,  ..., 0, 3, 1],
        ...,
        [2, 0, 1,  ..., 2, 1, 4],
        [2, 3, 3,  ..., 0, 2, 0],
        [4, 2, 3,  ..., 1, 4, 1]])
Beta param : torch.Size([5, 50])
alpha param : torch.Size([4])
theta_t param : torch.Size([25, 5])
W shape:  torch.Size([25, 50])
z shape:  torch.Size([50, 25])


RuntimeError: The size of tensor a (50) must match the size of tensor b (25) at non-singleton dimension 1